In [ ]:
import tqdm
import multiprocessing
import numpy as np
import os, glob, json
import copy

# Set seed
seed = 47
np.random.seed(seed)

set_ = 'train'
path = f'/data/mint/DPM_Dataset/ffhq_256_with_anno/ffhq_256/{set_}/'
img_path = glob.glob(path + '*.jpg')
img_name = [name.split('/')[-1] for name in img_path]
# print(img_name[:100])

if len(img_name) % 2 != 0:
    img_name.append(img_name[0])

rep_size = 20
sampling = {'pair':{}}
counter = 0
source_occur = np.array([0] * len(img_name))
target_occur = np.array([0] * len(img_name))

sampling_img = img_name.copy()
for i in range(rep_size):
    np.random.shuffle(sampling_img)
    # np.random.permutation(sampling_img)
    if i == 0:
        pass
    else: 
        # print(prev_sampling_img[:10], sampling_img[:10])
        # print(np.array(prev_sampling_img) == np.array(sampling_img))
        # print(np.where(np.array(prev_sampling_img) == np.array(sampling_img)))
        # print(np.array(prev_sampling_img) != np.array(sampling_img))
        # print(np.all(np.array(prev_sampling_img) != np.array(sampling_img)))
        # for k in np.where(np.array(prev_sampling_img) == np.array(sampling_img))[0]:
        #     print(prev_sampling_img[k], sampling_img[k])
        where_static = np.where(np.array(prev_sampling_img) == np.array(sampling_img))[0]
        where_static = where_static[1:] - where_static[:-1]
        if np.any(where_static == 1):
            raise ValueError('There is a static pair')
    
    # old_sampling_img should not be the same as sampling_img
    for j in range(0, len(sampling_img), 2):
        sampling['pair'][f'pair{counter}'] = {'src':sampling_img[j],
                                       'dst':sampling_img[j + 1]
                                    }
        source_occur[int(sampling_img[j].split('.')[0])] += 1
        target_occur[int(sampling_img[j + 1].split('.')[0])] += 1
        counter += 1
    prev_sampling_img = copy.deepcopy(sampling_img)
        
        
sampling['misc'] = {'rep_size':rep_size,
                'seed':seed,
                'set_':set_,
                'source_occur':source_occur.tolist(),
                'target_occur':target_occur.tolist(),
            }
json.dump(sampling, open(f'./generated_dataset_seed={seed}.json', 'w'), indent=4)


# Examine sampling coverage

In [ ]:
print("="*50)
print("Source occurence")
print("Min-Max occurence of source image : ", np.min(source_occur), np.max(source_occur))
print("Mean occurence of source image : ", np.mean(source_occur))
print("Std occurence of source image : ", np.std(source_occur))
print("Unique occurence of source image : ", np.unique(source_occur))
print("="*50)

print("Target occurence")
print("Min-Max occurence of target image : ", np.min(target_occur), np.max(target_occur))
print("Mean occurence of target image : ", np.mean(target_occur))
print("Std occurence of target image : ", np.std(target_occur))
print("Unique occurence of target image : ", np.unique(target_occur))
print(f"Number of occurence <>? : ", len(np.where(target_occur == 0)[0]))
print("="*50)

In [ ]:
# Plot histogram of occurence
import plotly.graph_objects as go


# Create traces
fig = go.Figure()

fig.add_trace(go.Scatter(x=list(range(len(source_occur))), y=source_occur,
                    mode='lines+markers',
                    name='Source'))
fig.add_trace(go.Scatter(x=list(range(len(target_occur))), y=target_occur,
                    mode='lines+markers',
                    name='Target'))

fig.show()


# Try load file

In [2]:
import time, json
f = open('./generated_dataset_seed=47.json', 'r')
timing = time.time()
sample_pairs = json.load(f)['pair']
print(f"Time to load json : {time.time() - timing}")

Time to load json : 0.7823045253753662
